In [1]:
!rm hawaii.sqlite

In [2]:
# Import SQL Alchemy
import sqlalchemy
from sqlalchemy import create_engine, MetaData

# Import PyMySQL (Not needed if mysqlclient is installed)
import pymysql
pymysql.install_as_MySQLdb()

# Import and establish Base for which classes will be constructed
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, DateTime

# Create Database Connection
# ----------------------------------
# Establish Connection to MySQL
engine = create_engine('sqlite:///hawaii.sqlite')


In [3]:
# Create Measurements and Stations classes
# ----------------------------------
class Measurements(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True,autoincrement=True)
    station = Column(String(255))
    date= Column(String(10))
    prcp = Column(Float)
    tobs = Column(Integer)

class Stations(Base):
    __tablename__ = 'stations'
    station = Column(String(255), primary_key=True)
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [4]:
Base.metadata.create_all(engine)

In [5]:
Base.metadata.tables

immutabledict({'measurements': Table('measurements', MetaData(bind=None), Column('id', Integer(), table=<measurements>, primary_key=True, nullable=False), Column('station', String(length=255), table=<measurements>), Column('date', String(length=10), table=<measurements>), Column('prcp', Float(), table=<measurements>), Column('tobs', Integer(), table=<measurements>), schema=None), 'stations': Table('stations', MetaData(bind=None), Column('station', String(length=255), table=<stations>, primary_key=True, nullable=False), Column('name', String(length=255), table=<stations>), Column('latitude', Float(), table=<stations>), Column('longitude', Float(), table=<stations>), Column('elevation', Float(), table=<stations>), schema=None)})

In [6]:
# Load the cleaned csv file into a pandas dataframe
# YOUR CODE HERE

import pandas as pd

measurements_df = pd.read_csv("Cleaned/clean_hawaii_measurements.csv")

stations_df = pd.read_csv("Cleaned/clean_hawaii_stations.csv")

# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
# YOUR CODE HERE

measurements_data = measurements_df.to_dict(orient='records')
stations_data = stations_df.to_dict(orient='records')

# Use MetaData from SQLAlchemy to reflect the tables
# YOUR CODE HERE

metadata = MetaData(bind=engine)

metadata.reflect()

In [7]:
measurements_table = sqlalchemy.Table('measurements', metadata, autoload=True)
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [8]:
conn = engine.connect()

conn.execute(measurements_table.delete())
conn.execute(stations_table.delete())

In [9]:
conn.execute(measurements_table.insert(),measurements_data)
conn.execute(stations_table.insert(),stations_data)

In [10]:
# Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [11]:
measurement_count = session.query(Measurements.id,Measurements.station).count()

measurement_count

18103

In [12]:
station_count = session.query(Stations.station).count()

station_count

9